<center style="font-size: 26px"> <b>Проверка уровня знаний Python</b></center>

In [ ]:
# обеспечиваем совместимость с Python 2 и 3
from __future__ import (absolute_import, division, print_function, unicode_literals)

# отключаем предупреждения дистрибутива Anaconda
import warnings
warnings.simplefilter('ignore')

**Описание датасета:**   
[Датасет вакансий с платформы HH.ru на Kaggle.com](https://www.kaggle.com/datasets/pavfedotov/heaadhunter-vacancies?resource=download&select=df2021-08-03.csv) 

Нужный файл - df2021-08-03.csv.

- *vacancy* - наименование вакансии
- *url* - ссылка на вакансию
- *created* - дата и время создания
- *has_test* - наличие тестового задания в вакансии
- *salary_from* - нижняя граница значения заработной платы
- *salary_to* - верхняя граница значения заработной платы
- *currency* - валюта заработной платы
- *experience* - требуемый опыт
- *schedule* - тип рабочего графика
- *skills* - требуемые навыки
- *employer* - наименование работодателя
- *area* - наименование города
- *description* - описание вакансии

#### Загрузите датасет

In [3]:
# ВАШ КОД ЗДЕСЬ
import numpy as np
import pandas as pd
st = r'./data_biocad.csv'
dataset=pd.read_csv(st.strip("‪u202a"), on_bad_lines='skip')
print(dataset)

                                                vacancy  \
0                 Backend/Full-stack developer (python)   
1     Бэкенд-разработчик (Python) / Middle Python / ...   
2          Fullstack Middle Python (Django) / VueJS Dev   
3                                      Python Developer   
4                                QA Automation (Python)   
...                                                 ...   
1695  Middle Software Developer C++/Python (relocati...   
1696                                   Python Tech Lead   
1697  Senior Software Developer C++/Python (relocati...   
1698               Full-stack Developer (React, Python)   
1699  Веб-разработчик Full stack (Java,JS, Python, X...   

                                                    url  \
0     https://hh.ru/applicant/vacancy_response?vacan...   
1     https://hh.ru/applicant/vacancy_response?vacan...   
2     https://hh.ru/applicant/vacancy_response?vacan...   
3     https://hh.ru/applicant/vacancy_response?vacan...

### Практические задания:

#### 1. Изучите методы для сбора данных курсов валют в [API Центрального банка Российской Федерации](https://www.cbr.ru/development/SXML/) . Используя запросы к API, приведите значения заработной платы к рублям одним из вариантов: 1) простой вариант - на текущую дату  2) вариант посложнее - на дату создания (created).

In [4]:
# ВАШ КОД ЗДЕСЬ
import requests
#Чтобы получить данные на текущую дату достаточно отправить запрос без даты
res=requests.get('https://www.cbr.ru/scripts/XML_daily.asp?')
res_xml=pd.read_xml(res.content, encoding='Windows-1251')
#"encoding" решает проблему нечитаемости названия валюты
dataset_sal_now=dataset.copy()
dataset_sal_created=dataset.copy()
#предыдущие две строки создаются, чтобы была возможность вывести весь датасет с измененной заработной платой на текущую дату и на дату создания соответственно
size_dataset=len(dataset.currency)
for k in range(size_dataset):
    data_curr=dataset.currency[k]
    data_create=dataset.created[k]
    # проверяем, что зарплата первоначально не в рублях
    if data_curr!='RUR':
        # в s формируется дата создания, именно так потому что первоначально дата дана в формате 'YY-MM-DDT{time}'
        date_for_requests="date_req="+(data_create)[8:10]+"/"+(data_create)[5:7]+"/"+(data_create)[:4]
        #проверка на тип валюты, отсекает значения nan
        if type(data_curr)==str:
            #проверка что зарплата указана(хоть что-то из нижней или верхней границы)
            if np.isnan(dataset.salary_from[k])==False:
                dataset_sal_now.loc[k, 'salary_from']=dataset.salary_from[k]*float((res_xml.VunitRate[list(res_xml.CharCode).index(data_curr)]).replace(",","."))
                #переводим зарплату в рубли 
                res_create=requests.get(f'https://www.cbr.ru/scripts/XML_daily.asp?{date_for_requests}')
                #запрос на дату создания
                res_xml_create=pd.read_xml(res_create.content, encoding='Windows-1251')
                
                dataset_sal_created.loc[k, 'salary_from']=dataset.salary_from[k]*float((res_xml_create.VunitRate[list(res_xml_create.CharCode).index(data_curr)]).replace(",","."))
                #loc-позволяет изменить значение в ячейке только в заданном датасете (также для этого здесь - dataset_sal_now=dataset.copy() используется copy())
            if np.isnan(dataset.salary_to[k])==False:
                dataset_sal_now.loc[k, 'salary_to']=dataset.salary_to[k]*float((res_xml.VunitRate[list(res_xml.CharCode).index(data_curr)]).replace(",","."))
                res_now=requests.get(f'https://www.cbr.ru/scripts/XML_daily.asp?{date_for_requests}')
                res_xml_now=pd.read_xml(res.content, encoding='Windows-1251')
                dataset_sal_created.loc[k, 'salary_to']=dataset.salary_to[k]*float((res_xml_create.VunitRate[list(res_xml_create.CharCode).index(data_curr)]).replace(",","."))
            dataset_sal_now.loc[k, 'currency']='RUR'
            dataset_sal_created.loc[k, 'currency']='RUR'
            #Поменяли валюту на рубли
print(dataset_sal_created)
#выведем, например, датасет с заработной платой, переведенной в рубли на дату создания вакансии


                                                vacancy  \
0                 Backend/Full-stack developer (python)   
1     Бэкенд-разработчик (Python) / Middle Python / ...   
2          Fullstack Middle Python (Django) / VueJS Dev   
3                                      Python Developer   
4                                QA Automation (Python)   
...                                                 ...   
1695  Middle Software Developer C++/Python (relocati...   
1696                                   Python Tech Lead   
1697  Senior Software Developer C++/Python (relocati...   
1698               Full-stack Developer (React, Python)   
1699  Веб-разработчик Full stack (Java,JS, Python, X...   

                                                    url  \
0     https://hh.ru/applicant/vacancy_response?vacan...   
1     https://hh.ru/applicant/vacancy_response?vacan...   
2     https://hh.ru/applicant/vacancy_response?vacan...   
3     https://hh.ru/applicant/vacancy_response?vacan...

#### 2. Найдите вакансию с самой высокой заработной платой, где подойдет 4 года опыта работы и не нужно проходить тестовое задание.

In [5]:
max_salary=0
row_for_answer=0
for row in range(size_dataset):
    experiense=dataset_sal_created.experience[row].split()
    #проверяем, что не нужно проходить тестовое задание
    if dataset_sal_created.has_test[row]==False:
        #Исключаем вакансии с отсутствием опыта
        if "От" in experiense and "до" in experiense:
            min_experience=experiense[experiense.index("От")+1]
            max_experience=experiense[experiense.index("до")+1]
            #Проверяем, что подойдет 4 года опыта и указана зарплата (нижняя или верхняя граница)
            if int(min_experience)<=4 and int(max_experience)>=4 and (np.isnan(dataset_sal_created.salary_to[row])==False or np.isnan(dataset_sal_created.salary_from[row])==False):
                salary=max(dataset_sal_created.salary_to[row], dataset_sal_created.salary_from[row])
                if salary>max_salary:
                    #Ищем строку в датасете подходящую под условия и с наибольшей заработной платой, переведенной в рубли на момент создания вакансии
                    max_salary=salary
                    row_for_answer=row
print(dataset_sal_created.loc[row_for_answer])


        
    


vacancy                     Full-stack Разработчик React, Python
url            https://hh.ru/applicant/vacancy_response?vacan...
created                                 2021-08-01T03:53:23+0300
has_test                                                   False
salary_from                                             200000.0
salary_to                                               500000.0
currency                                                     RUR
experience                                         От 3 до 6 лет
schedule                                        Удаленная работа
skills                                             React;Python;
employer                                                MINT NFT
area                                                 Владивосток
description    MINT NFT, Inc. - компания занимается разработк...
Name: 140, dtype: object


#### 3. Сделайте рейтинг (топ-30) навыков по всем вакансиям.

In [6]:
# ВАШ КОД ЗДЕСЬ
SKILLS=[]
count_skills=[]
for row in range(size_dataset):
    skill_one_vacancy=dataset.skills[row]
    if type(skill_one_vacancy)!=float:
        #проверка, что навыки в вакансии указаны
        skill_one_vacancy=dataset.skills[row].split(";")
        #Разбиваем все указанные навыки по ;
        for cell in skill_one_vacancy:
            if cell in SKILLS:
                #Если этот навык уже встречался, то просто добавляем единицу к его количеству упоминаний
                count_skills[SKILLS.index(cell)]+=1
            elif cell not in SKILLS and cell!="":
                #Если не встречался, добавляем в список
                SKILLS.append(cell)
                count_skills.append(1)
count_skills, SKILLS = zip(*sorted(zip(count_skills, SKILLS)))
#Сортируем массивы относительно друг друга
SKILLS=SKILLS[::-1]
count_skills=count_skills[::-1]
#Переворачиваем, чтобы в начале были самые популярные навыки
for i in range(30):
    print(str(i+1)+".", SKILLS[i])

    

1. Python
2. Git
3. SQL
4. PostgreSQL
5. Django Framework
6. Linux
7. Flask
8. Docker
9. JavaScript
10. Английский язык
11. MySQL
12. ООП
13. MongoDB
14. Redis
15. REST
16. RabbitMQ
17. Django
18. HTML
19. CSS
20. Atlassian Jira
21. C++
22. API
23. MS SQL
24. Java
25. Базы данных
26. Работа в команде
27. AWS
28. Pytest
29. Celery
30. aiohttp


#### 4. Cоставьте топ-10 городов с наибольшей средней заработной платой по вакансиям тестировщика (QA).

In [7]:
city_top=[]
averege_salary=[]
def averege_salary_in_city(area, dataset_area, dataset_sal_to, dataset_sal_from, vacancy, size_dataset, row_start):
    count=0
    sum_salary=0
    for row in range(row_start, size_dataset):
        if dataset_area[row]==area and "QA" in vacancy[row]:
            #Если указан и верхний и нижний потолок в вакансии, то я считаю среднее по вакансии, если указан только один из двух параметров, то беру только его
            if np.isnan(dataset_sal_to[row])==False and np.isnan(dataset_sal_from[row])==False:
                sum_salary+=(dataset_sal_to[row]+dataset_sal_from[row])/2
                count+=1
            elif np.isnan(dataset_sal_to[row])==False:
                sum_salary+=dataset_sal_to[row]
                count+=1
            elif np.isnan(dataset_sal_from[row])==False:
                sum_salary+=dataset_sal_from[row]
                count+=1
    if sum_salary!=0:
        return sum_salary/count
    else:
        return 0
for row in range(size_dataset):
    area=dataset_sal_created.area[row]
    #Проверяем, что этот город мы еще не встречали, и что в этой вакансии требуется тестировщик
    if  area not in city_top and "QA" in dataset_sal_created.vacancy[row]:
        city_top.append(area)
        #Функция рассчитывает среднюю зп по городу, проходясь только по следующим после текущей строчкам (очевидно, что предыдущие нам просто не подходят)
        averege_salary.append(averege_salary_in_city(area,
                                                    dataset_sal_created.area,
                                                    dataset_sal_created.salary_to,
                                                    dataset_sal_created.salary_from,
                                                    dataset_sal_created.vacancy, 
                                                    size_dataset, 
                                                    row))
averege_salary, city_top = zip(*sorted(zip(averege_salary, city_top)))
averege_salary=averege_salary[::-1]

city_top=city_top[::-1]

for k in range(10):
    if k<len(averege_salary):
        if averege_salary[k]==0:
            print(str(k+1)+'. '+city_top[k]+'\n'+"Зарплата не была указана"+"\n")
        else:
            print(str(k+1)+'. '+city_top[k]+'\n'+'Средняя заработная плата - '+str(int(averege_salary[k]))+"\n")
    else:
        print("Нашлось только "+str(k)+" городов, где были вакансии тестировщиков")
        break


1. Тамбов
Средняя заработная плата - 200000

2. Москва
Средняя заработная плата - 190555

3. Санкт-Петербург
Средняя заработная плата - 180555

4. Казань
Средняя заработная плата - 90000

5. Саратов
Зарплата не была указана

6. Самара
Зарплата не была указана

7. Новосибирск
Зарплата не была указана

Нашлось только 7 городов, где были вакансии тестировщиков


#### 5. Напишите функцию для поиска вакансий по переданным в нее значениям заработной платы, наличия тестового задания, наличия заданного текста в наименовании вакансии, наличия заданного навыка. В результате выведите количество найденных вакансий, среднюю заработную плату по найденным вакансиям и список ссылок найденных вакансий. 

In [8]:
# ВАШ КОД ЗДЕСЬ

def search_vacancy(salary, test, description, skill):
    sum_salary=0
    count=0
    global dataset_sal_created, size_dataset, result
    for vacancy in range(size_dataset):
        sal_to=dataset_sal_created.loc[vacancy,'salary_to']
        sal_from=dataset_sal_created.loc[vacancy,'salary_from']
        availability_of_test=dataset_sal_created.loc[vacancy,'has_test']
        description_check=dataset_sal_created.loc[vacancy,'description']
        skills_check=dataset_sal_created.loc[vacancy,'skills']
        if np.isnan(sal_to)==False and np.isnan(sal_from)==False:
            if salary>=sal_from and salary<=sal_to:
                if availability_of_test==test:
                    if description in description_check:
                        if skill in skills_check:
                            sum_salary+=(sal_from+sal_to)/2
                            count+=1
                            result.append(i)
        elif np.isnan(sal_to)==False:
            if salary<=sal_to:
                if availability_of_test==test:
                    if description in description_check:
                        if skill in skills_check:
                            sum_salary+=sal_to
                            count+=1
                            result.append(i)
        elif np.isnan(sal_from)==False:
            if salary>=sal_from:
                if availability_of_test==test:
                    if description in description_check:
                        if skill in skills_check:
                            sum_salary+=sal_from
                            count+=1
                            result.append(i)
    if count!=0:
        sum_salary/=count
    return "Количество найденных вакансий - "+ str(count) + ";  Средняя зарплата - "+str(sum_salary)+"\n"
result=[]
print(search_vacancy(210000, False, "NFT", "Python"))
for i in result:
    print(dataset_sal_created.url[i], "\n")
                    
        

Количество найденных вакансий - 8;  Средняя зарплата - 331250.0

https://hh.ru/applicant/vacancy_response?vacancyId=46325809 

https://hh.ru/applicant/vacancy_response?vacancyId=46325809 

https://hh.ru/applicant/vacancy_response?vacancyId=46325809 

https://hh.ru/applicant/vacancy_response?vacancyId=46325809 

https://hh.ru/applicant/vacancy_response?vacancyId=46325809 

https://hh.ru/applicant/vacancy_response?vacancyId=46325809 

https://hh.ru/applicant/vacancy_response?vacancyId=46325809 

https://hh.ru/applicant/vacancy_response?vacancyId=46325809 



#### *Необязательно*. Усложненный вариант - сделать атрибуты функции опциональными, чтобы иметь возможность фильтрации вакансий по любой комбинации условий.

In [9]:
# ВАШ КОД ЗДЕСЬ
#Функция проходится по всем параметрам, если какой-то из них не удовлетворяет поиску, номер вакансии удаляется из результата

def filter(vacancy=None, url=None,  created=None, has_test=None, salary_from=None, salary_to=None, currency=None, experience=None, schedule=None, skills=None, employer=None, area=None, description=None):
    global dataset_sal_created, size_dataset
    result=[]
    for k in range(size_dataset):
        result.append(k)
    dataset=dataset_sal_created
    for row in range(size_dataset):
        flag=0
        if vacancy:
            if dataset.vacancy[row]!=vacancy:
                flag=1
                result.remove(row)
        if url and flag==0:
            if dataset.url[row]!=url:
                result.remove(row)  
                flag=1
        if created and flag==0:
            if dataset.created[row]!=created:
                result.remove(row)  
                flag=1
        if has_test and flag==0:
            if dataset.has_test[row]!=has_test:
                result.remove(row)  
                flag=1
        if salary_from and flag==0:
            if dataset.salary_from[row]>=salary_from or np.isnan(dataset.salary_from[row])==True:
                result.remove(row) 
                flag=1
        if salary_to and flag==0:
            if dataset.salary_to[row]<=salary_to or np.isnan(dataset.salary_to[row])==True:
                result.remove(row) 
                flag=1
        if currency and flag==0:
            if dataset.currency[row]!=currency:
                result.remove(row)
                flag=1
        if experience and flag==0:
            if dataset.experience[row]!=experience:
                result.remove(row)
                flag=1 
        if schedule and flag==0:
            if dataset.schedule[row]!=schedule:
                result.remove(row)
                flag=1
        if skills and flag==0:
            c=0
            SKILLS=dataset.skills[row]
            if type(SKILLS)!=float:
                SKILLS=SKILLS.split(";")
                
                
                for i in skills:
                    if i not in SKILLS:
                        c=1
                        break
                if c==1:
                    result.remove(row)
                    flag=1
            else:
                result.remove(row)
                flag=1
            
        if employer and flag==0:
            if dataset.employer[row]!=employer:
                result.remove(row)
                flag=1
        if area and flag==0:
            if dataset.area[row]!=area:
                result.remove(row)
                flag=1
        if description and flag==0:
            if dataset.description[row]!=description:
                result.remove(row)
    return result   
result=filter(skills=['Python','MySQL', 'Rest API'])                
if result!=[]:
    for i in result:
        print(dataset_sal_created.loc[i])



vacancy                             Backend-разработчик (Python)
url            https://hh.ru/applicant/vacancy_response?vacan...
created                                 2021-07-30T11:51:04+0300
has_test                                                   False
salary_from                                             130000.0
salary_to                                               250000.0
currency                                                     RUR
experience                                    От 1 года до 3 лет
schedule                                             Полный день
skills         Linux;MySQL;Rest API;GraphQL;ООП;Git;SQL;REST;...
employer                                      Авгуро Технолоджис
area                                                      Москва
description    Авгуро Технолоджис — мультибрендовый разработч...
Name: 1317, dtype: object
